In [20]:
#same as AoC_2015_22 just done in OOP
#https://adventofcode.com/2015/day/22
#https://www.reddit.com/r/adventofcode/comments/3xspyl/day_22_solutions/

In [25]:
try:
    from IPython import get_ipython
    get_ipython().magic('reset -f')  
except:
    pass
from IPython import get_ipython
ipython = get_ipython().magic
import os
os.chdir('/Users/markprosser/Desktop/PythonScripts')
import MCP_functions as MCP
import numpy as np
import matplotlib.pyplot as plt
import sys
import random
plt.close("all")
import itertools
import warnings
warnings.simplefilter(action='ignore')

from random import randint

In [68]:
startbosshp = 51
startbossdamage = 9
startbossarmor = 0

startwizardhp = 50
startwizardarmor = 0
startwizarddamage = 0
startwizardmana = 500
startwizardst = 0 #shield turns
startwizardpt = 0 #poison
startwizardrt = 0 #recharge
startspentmana = 0

In [100]:
class Boss:
    def __init__(self, hp, damage, armor):
        self.hp = hp
        self.damage = damage
        self.armor = armor
        
    def calc_actdamage(self, wizardarmor):
        self.actdamage = self.damage - wizardarmor
        if (self.actdamage < 1):
            self.actdamage = 1
            
class Wizard:
    def __init__(self, hp, damage, armor, mana, st, pt, rt, spentmana):
        self.hp = hp
        self.damage = damage
        self.armor = armor
        self.mana = mana
        self.st = st
        self.pt = pt
        self.rt = rt
        self.spentmana = spentmana
            
 


In [72]:
def list_mana(wizardmana):
    if (wizardmana < 73):
        listmana = [0]
    elif (wizardmana < 113):
        listmana = [0,1]
    elif (wizardmana < 173):
        listmana = [0,1,2]
    elif (wizardmana < 229):
        listmana = [0,1,2,3]
    elif (wizardmana >= 229):
        listmana = [0,1,2,3,4]
    return listmana

def what_randi(st, pt, rt, wizardmana):        
    list0 = [0,1,2,3,4]
    list1s = [0,1,3,4]
    list1p = [0,1,2,4]
    list1r = [0,1,2,3]
    list2sp = [0,1,4]
    list2pr = [0,1,2]
    list2sr = [0,1,3]
    list3 = [0,1]

    if (st==0) & (pt==0) & (rt==0):
        listmana = list_mana(wizardmana)
        randi = random.choice(list(set(list0).intersection(listmana)))           
    elif (st!=0) & (pt==0) & (rt==0):
        listmana = list_mana(wizardmana)
        randi = random.choice(list(set(list1s).intersection(listmana))) 
    elif (st==0) & (pt!=0) & (rt==0):
        listmana = list_mana(wizardmana)
        randi = random.choice(list(set(list1p).intersection(listmana))) 
    elif (st==0) & (pt==0) & (rt!=0):
        listmana = list_mana(wizardmana)
        randi = random.choice(list(set(list1r).intersection(listmana))) 
    elif (st!=0) & (pt!=0) & (rt==0):
        listmana = list_mana(wizardmana)
        randi = random.choice(list(set(list2sp).intersection(listmana))) 
    elif (st==0) & (pt!=0) & (rt!=0):
        listmana = list_mana(wizardmana)
        randi = random.choice(list(set(list2pr).intersection(listmana))) 
    elif (st!=0) & (pt==0) & (rt!=0):
        listmana = list_mana(wizardmana)
        randi = random.choice(list(set(list2sr).intersection(listmana))) 
    elif (st!=0) & (pt!=0) & (rt!=0):
        listmana = list_mana(wizardmana)
        randi = random.choice(list(set(list3).intersection(listmana))) 
    return randi

def update_turns(st, pt, rt, wizardarmor, bosshp, wizardmana):
    if (st > 0):
        wizardarmor = 7
        st -= 1
    elif (st == 0):
        wizardarmor = 0     
    if (pt > 0):
        bosshp -= 3
        pt -= 1
    elif (pt == 0):
        pass
    if (rt > 0):
        rt -= 1
        wizardmana += 101
    elif (rt == 0):
        pass
    return st, pt, rt, wizardarmor, bosshp, wizardmana

def update_wiz_boss(randi, st, pt, rt, wizardarmor, bosshp, wizardmana, wizardhp, spentmana):
    if randi == 0:
        wizardmana -= 53
        spentmana += 53
        bosshp -= 4
        st, pt, rt, wizardarmor, bosshp, wizardmana = update_turns(st, pt, rt, wizardarmor, bosshp, wizardmana)
    elif randi == 1:
        wizardmana -= 73
        spentmana += 73
        bosshp -= 2
        wizardhp += 2
        st, pt, rt, wizardarmor, bosshp, wizardmana = update_turns(st, pt, rt, wizardarmor, bosshp, wizardmana)
    elif randi == 2:
        st = 6
        wizardmana -= 113
        spentmana += 113
        st, pt, rt, wizardarmor, bosshp, wizardmana = update_turns(st, pt, rt, wizardarmor, bosshp, wizardmana)
    elif randi == 3:
        pt = 6
        wizardmana -= 173
        spentmana += 173
        st, pt, rt, wizardarmor, bosshp, wizardmana = update_turns(st, pt, rt, wizardarmor, bosshp, wizardmana)   
    elif randi == 4:
        rt = 5
        wizardmana -= 229
        spentmana += 229
        st, pt, rt, wizardarmor, bosshp, wizardmana = update_turns(st, pt, rt, wizardarmor, bosshp, wizardmana)
    return st, pt, rt, wizardarmor, bosshp, wizardmana, wizardhp, spentmana

In [94]:
bestspentmanalist = []
bestlist = []
lose_no = 0
win_no = 0
bestspentmana = 99999


for i in range(0, 100000):
    t=1
    templist = [] #saving the move order in case it is good
    #create instances
    boss_i = Boss(startbosshp, startbossdamage, startbossarmor)
    wiz_i = Wizard(startwizardhp, startwizarddamage, startwizardarmor, startwizardmana, \
                       startwizardst, startwizardpt, startwizardrt, startspentmana)
    randi = what_randi(wiz_i.st, wiz_i.pt, wiz_i.rt, wiz_i.mana) #wizard 1st attack
    templist.append(randi)
    wiz_i.st, wiz_i.pt, wiz_i.rt, wiz_i.armor, boss_i.hp, wiz_i.mana, wiz_i.hp, \
                        wiz_i.spentmana = update_wiz_boss(randi, wiz_i.st, wiz_i.pt, \
                        wiz_i.rt, wiz_i.armor, boss_i.hp, wiz_i.mana, wiz_i.hp, \
                        wiz_i.spentmana)
    boss_i.calc_actdamage(wiz_i.armor)
    wiz_i.hp -= boss_i.actdamage #boss' 1st attack
    while (boss_i.hp > 0) & (wiz_i.hp > 0) & (wiz_i.mana > 52): 
        t += 1
        randi = what_randi(wiz_i.st, wiz_i.pt, wiz_i.rt, wiz_i.mana)
        templist.append(randi)
        wiz_i.st, wiz_i.pt, wiz_i.rt, wiz_i.armor, boss_i.hp, wiz_i.mana, wiz_i.hp, \
                        wiz_i.spentmana = update_wiz_boss(randi, wiz_i.st, wiz_i.pt, \
                        wiz_i.rt, wiz_i.armor, boss_i.hp, wiz_i.mana, wiz_i.hp, \
                        wiz_i.spentmana)
        boss_i.calc_actdamage(wiz_i.armor) 
        wiz_i.hp -= boss_i.actdamage #boss's attack 
        if (wiz_i.mana < 53) & (boss_i.hp > 0): #if wizard becomes sitting duck
            wiz_i.hp = -999
    
    if (wiz_i.hp > boss_i.hp): #if you win
        win_no += 1
        if (wiz_i.spentmana < bestspentmana): #if you win AND cheap spend
            bestspentmana = wiz_i.spentmana
            bestspentmanalist.append(wiz_i.spentmana)
            bestlist.append(templist)
        else:
            templist = []
    elif (boss_i.hp > wiz_i.hp):
        lose_no += 1
        templist = []
        
print('bestspentmana= ',bestspentmana) 
print('bestspentmanalist= ',bestspentmanalist) 
print('bestlist= ',bestlist) 
print('lose_no= ',lose_no) 
print('win_no= ',win_no) 

bestspentmana=  1334
bestspentmanalist=  [1494, 1388, 1348, 1334]
bestlist=  [[4, 2, 0, 0, 0, 4, 3, 0, 2, 1, 0, 1, 3, 0], [4, 3, 2, 1, 0, 0, 4, 3, 1, 0, 2, 0], [3, 4, 2, 0, 0, 0, 4, 3, 0, 2, 0, 0], [2, 4, 0, 3, 0, 0, 4, 0, 2, 0, 0, 0, 0, 0]]
lose_no=  972136
win_no=  27722


In [99]:
from pprint import pprint
pprint(vars(wiz_i))

{'armor': 0,
 'damage': 0,
 'hp': -7,
 'mana': 158,
 'pt': 5,
 'rt': 0,
 'spentmana': 847,
 'st': 0}


-999